# Experiments with backtesting.py
This notebook is to explore the `backtesting.py` to evaluate strategy:
1. Based on conventional TA (talib etc.)
2. See how can we introduce ML model as TA

Install ta-lib on Mac with brew first `brew install ta-lib` and then `pip install TA-Lib`

In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import yfinance as yf
import talib 
from utils import get_stock_data

/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### 1. Standard Examples
First, use standard example; where goal is just to see how we can use the library. In the following, we will use SMA as an indicator which is already available in `backtesting.py`. We will also use stop loss and take profit to limit the exposure and to ensure risk-reward ratio.

In [2]:
class SmaCrossWithSLTP(Strategy):
    short_window = 10  # Fast SMA
    long_window = 20   # Slow SMA
    stop_loss_pct = 0.02   # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit
    
    # All the necessary indicator needs to define in the init method
    def init(self):
        self.sma_short = self.I(SMA, self.data.Close, self.short_window)
        self.sma_long = self.I(SMA, self.data.Close, self.long_window)
    
    # It defines the logic of the strategy
    def next(self):
        price = self.data.Close[-1]  # Current price
        if crossover(self.sma_short, self.sma_long):  # Buy condition
            stop_loss = price * (1 - self.stop_loss_pct)
            take_profit = price * (1 + self.take_profit_pct)
            self.buy(sl=stop_loss, tp=take_profit)  # Order with SL and TP
        
        elif crossover(self.sma_long, self.sma_short):  # Sell condition
            self.position.close()  # Close existing position

# Load sample data which comes with the library to test
df = GOOG

# Run Backtest
bt = Backtest(df, SmaCrossWithSLTP, cash=10000, commission=0.002)
stats = bt.run()
print(stats)
# bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     9.82309
Equity Final [$]                  11532.01791
Equity Peak [$]                   12667.87271
Commissions [$]                    2041.36509
Return [%]                           15.32018
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                     1.68634
Volatility (Ann.) [%]                 6.32304
CAGR [%]                              1.15945
Sharpe Ratio                           0.2667
Sortino Ratio                         0.40031
Calmar Ratio                          0.14875
Alpha [%]                            -4.51391
Beta                                  0.03266
Max. Drawdown [%]                   -11.33648
Avg. Drawdown [%]                    -3.53535
Max. Drawdown Duration      742 days 00:00:00
Avg. Drawdown Duration      172 days 00:00:00
# Trades                          

## 2. Integrating other indicators from TA-LIB
Now, moving to `ta-lib` for more indicators. In the following, we will use RSI along with SL and TP.

In [3]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14    # RSI period
    rsi_buy = 30       # RSI oversold threshold
    rsi_sell = 70      # RSI overbought threshold
    stop_loss_pct = 0.02  # 2% stop-loss
    take_profit_pct = 0.05  # 5% take-profit

    def init(self):    
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Entry Condition: RSI below 30 & SMA crossover
        if self.rsi[-1] < self.rsi_buy :
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  # Stop-Loss 2% below entry
            tp = price * (1 + self.take_profit_pct)  # Take-Profit 5% above entry
            self.buy(sl=sl, tp=tp)  # Place order with SL & TP

        # Exit Condition: RSI above 70 OR SMA crossover in opposite direction
        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

df = get_stock_data("MSFT")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# Save HTML file for plots for in-browser visualization
#bt.plot(filename="rsi_strategy.html")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
Backtest.run:   0%|          | 0/527 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=18: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=183: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=184: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=185: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                   

Start                     2022-01-03 00:00:00
End                       2024-02-29 00:00:00
Duration                    787 days 00:00:00
Exposure Time [%]                     3.32103
Equity Final [$]                  10442.84147
Equity Peak [$]                   10604.61397
Commissions [$]                     242.41337
Return [%]                            4.42841
Buy & Hold Return [%]                42.38855
Return (Ann.) [%]                     2.03511
Volatility (Ann.) [%]                 4.34213
CAGR [%]                              1.39716
Sharpe Ratio                          0.46869
Sortino Ratio                         0.81076
Calmar Ratio                          0.40808
Alpha [%]                             3.54798
Beta                                  0.02077
Max. Drawdown [%]                     -4.9871
Avg. Drawdown [%]                    -3.24108
Max. Drawdown Duration      527 days 00:00:00
Avg. Drawdown Duration      265 days 00:00:00
# Trades                          

## 3. Integrating a black-box indicator (e.g., ML model)
Now we will add a black-box function which can be an ML model which can provide BUY or SELL signal. We will pass the dataframe containing the features and function can use this to predict the position. This then can be used in `next` function.

In [4]:
def dummy_ml_model(df):
    """
    Simulated ML model that predicts BUY signals. We don't consider SELL (short positions).
    :param df: DataFrame of last `N` rows (OHLCV features).
    :return: 1 if BUY signal, else 0.
    """
    # Simulated logic: Buy if the last row closes lower than the first row in the window
    if df.iloc[-1]['Close'] < df.iloc[0]['Close']:  
        return 1  # BUY Signal
    return 0  # No action

class MLTradingStrategy(Strategy):
    feature_window = 5 # window size for features 
    stop_loss_pct = 0.02 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            "Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = dummy_ml_model(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT") 

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
#bt.plot(filename="ml__strategy.html")

[*********************100%***********************]  1 of 1 completed
Backtest.run:   0%|          | 0/541 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=7: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=10: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=11: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=12: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/sit

Start                     2022-01-03 00:00:00
End                       2024-02-29 00:00:00
Duration                    787 days 00:00:00
Exposure Time [%]                    66.97417
Equity Final [$]                    5563.5934
Equity Peak [$]                   10037.51804
Commissions [$]                    2753.87426
Return [%]                          -44.36407
Buy & Hold Return [%]                26.06329
Return (Ann.) [%]                   -23.86149
Volatility (Ann.) [%]                19.60195
CAGR [%]                            -17.11767
Sharpe Ratio                          -1.2173
Sortino Ratio                        -1.25017
Calmar Ratio                         -0.39087
Alpha [%]                           -59.74531
Beta                                  0.59015
Max. Drawdown [%]                   -61.04773
Avg. Drawdown [%]                   -30.99982
Max. Drawdown Duration      778 days 00:00:00
Avg. Drawdown Duration      392 days 00:00:00
# Trades                          

## 4. Integrating ESN model from `1_esn_le2e.ipynb`
Here, our function will look for some threshold above which we can have BUY.

In [5]:
from utils import StockPricePredictor

predictor = StockPricePredictor(model_path="stock_esn_model_7.pkl", 
                                scaler_path="scaler_esn_model_7.pkl")

def if_buy_signal_with_esn(df, threshold=0.02):
    """
    Determines if a BUY signal should be generated based on the ESN model's prediction.
    
    Args:
        df (DataFrame): The dataframe containing the latest stock data.
        threshold (float): The minimum percentage increase required to trigger a BUY signal.
    
    Returns:
        int: 1 if BUY signal is triggered, otherwise 0.
    """
    predicted_close = predictor.predict(df)
    last_close = df.iloc[-1]['Close']
    
    # Check if predicted close is at least (1 + threshold)% higher than last close
    if predicted_close >= last_close * (1 + threshold):
        return 1  # BUY Signal
    
    return 0  # No action


class MLTradingStrategy(Strategy):
    feature_window = 7 # window size for features 
    stop_loss_pct = 0.025 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            #"Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = if_buy_signal_with_esn(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT") 

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# bt.plot(filename="esn_strategy.html")

Using Numpy backend.
[*********************100%***********************]  1 of 1 completed


Start                     2022-01-03 00:00:00
End                       2024-02-29 00:00:00
Duration                    787 days 00:00:00
Exposure Time [%]                      9.5941
Equity Final [$]                  11715.32159
Equity Peak [$]                   11715.32159
Commissions [$]                     561.38143
Return [%]                           17.15322
Buy & Hold Return [%]                26.06329
Return (Ann.) [%]                     7.63832
Volatility (Ann.) [%]                11.69376
CAGR [%]                               5.1999
Sharpe Ratio                           0.6532
Sortino Ratio                         1.30783
Calmar Ratio                          0.86888
Alpha [%]                            14.38134
Beta                                  0.10635
Max. Drawdown [%]                    -8.79098
Avg. Drawdown [%]                    -5.18562
Max. Drawdown Duration      287 days 00:00:00
Avg. Drawdown Duration       93 days 00:00:00
# Trades                          

### Summary
For same MSFT stock:
- Approach-2 (RSI)= 10442.84147
- Approach-3 (2 step trend)= 5563.5934
- Approach-4 (7 days to predicts via ESN)= 11715.32159

END OF NOTEBOOK